In [171]:
import pandas as pd
train = pd.read_csv('dataset/part2/ch4/train.csv')
test = pd.read_csv('dataset/part2/ch4/test.csv')

- 머신 러닝 프로세스
문제 정의 - 데이터 불러오기 - eda(탐색적 데이터 분석) - 데이터 전처리 - 검증 데이터 나누기 - 모델 학습 및 평가 - 예측 및 제출
- 데이터 전처리
concat X_train with X_validate - encoding - split

In [172]:
y_train = train.pop('Item_Outlet_Sales')
train.shape, y_train.shape, test.shape

((6818, 11), (6818,), (1705, 11))

In [173]:
from sklearn.preprocessing import LabelEncoder
def encode(df, cols):
    le = LabelEncoder()
    for col in cols:
        df[col] = le.fit_transform(df[col])
    return df

In [ ]:
cc = pd.concat([train, test], axis=0)
cols = cc.select_dtypes(include='O').columns
cols = list(cols)
cols.remove('Item_Identifier')

# todo: ???
labeled = encode(cc, cols)
train = labeled.iloc[:len(train)].copy()
display(train.isna().sum())
# test = labeled[len(train):].copy()

Item_Identifier                 0
Item_Weight                  1162
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [175]:
# train['Item_Weight'] = train['Item_Weight'].fillna(train['Item_Weight'].mean())
train.isna().sum()

Item_Identifier                 0
Item_Weight                  1162
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64